# Exercises

Try these exercises to test your skills on dates & times:

1. Using the New York City taxi data, calculate the length of each ride using the pickup & drop-off timestamps. Sort the query results from the longest ride to the shortest. Do you notice anything about the longest or shortest trips that you might want to ask city officials about?
2. Using the `AT TIME ZONE` keywords, write a query that displays the date & time for London, Johannesburg, Moscow, & Melbourne the moment January 1, 2100, arrives in New York City.
3. As a bonus challenge, calculate the correlation coefficient & r-squared values using trip time duration & the `total_amount` column in the New York City taxi data, which represents the total amount charged to passengers. Do the same with the `trip_distance` & `total_amount` columns. Limit the query to rides that last three hours or less.

---

# 1.

```
SET TIME ZONE 'US/Eastern';

SELECT *,
       tpep_dropoff_datetime - tpep_pickup_datetime
	   	   AS ride_length
FROM nyc_yellow_taxi_trips
ORDER BY ride_length DESC;
```

I don't know if I'm doing something wrong, but the longest ride lasts almost a full day: 24 hours. What's weird about it too is that you'd expect the amount paid for the ride, `total_amount`, to be very high for such a long ride, but its not.

<img src = "Exercise Images/Ride Length.png" width = "600" style = "margin:auto"/>

For example, the longest ride (time-wise) was from New York City Hall to New York City Hall. It's ride duration was 23 hours, 59 minutes, & 37 seconds, with a trip distance of 2.4. The amount paid for the ride as \\$14.30. The second longest ride duration was 23 hours, 59 minutes, & 31 seconds. The amount paid for the ride was \\$11.84, from New York City Hall to New York City Hall with a trip distance of 1.38.

The shortest duration ride was 1 hour, 27 minutes, & 15 seconds. The amount paid was \\$59.30, from City Hall to City Hall, with a trip distance of 14.25. The second shortest duration ride was 1 hour, 27 minutes, & 16 seconds. The amount paid for the ride was \\$354.85, with a trip distance of 58.91.

# 2.

```
SELECT *
FROM pg_timezone_names
WHERE name LIKE '%London%'
	OR name LIKE '%Johannesburg%'
	OR name LIKE '%Moscow%'
	OR name LIKE '%Melbourne%';

SELECT '2100-01-01 00:00:00 US/Eastern'
           AT TIME ZONE 'Europe/London'
           AS london_timezone,
       '2100-01-01 00:00:00 US/Eastern'
           AT TIME ZONE 'Europe/Moscow'
           AS moscow_timezone,
       '2100-01-01 00:00:00 US/Eastern'
           AT TIME ZONE 'Africa/Johannesburg'
           AS johannesburg_timezone,
       '2100-01-01 00:00:00 US/Eastern'
           AT TIME ZONE 'Australia/Melbourne'
           AS melboure_timezone;
```

<img src = "Exercise Images/Using AT TIME ZONE.png" width = "600" style = "margin:auto"/>

The moment it reaches January 1, 2100 in New York City, it will be 5 AM in London, 8 AM in Moscow, 7 AM in Johannesburg, & 4 PM in Melbourne.

# 3.

```
SELECT corr(total_amount,
           EXTRACT(epoch FROM tpep_dropoff_datetime -
           tpep_pickup_datetime)::integer)::numeric
           AS cost_duration_r,
       regr_r2(total_amount,
           EXTRACT(epoch FROM tpep_dropoff_datetime -
           tpep_pickup_datetime)::integer)::numeric
           AS cost_duration_r2
FROM nyc_yellow_taxi_trips
WHERE tpep_dropoff_datetime - 
   		  tpep_pickup_datetime < '3 hours';

SELECT corr(total_amount, trip_distance)::numeric
           AS cost_distance_r,
       regr_r2(total_amount, trip_distance)::numeric
           AS cost_distance_r2
FROM nyc_yellow_taxi_trips
WHERE tpep_dropoff_datetime -
          tpep_pickup_datetime < '3 hours';
```

The correlation between amount paid for the ride & the duration of the ride is denoted by `cost_duration_r`. The amount of variability in the amount paid for a ride described by the duration of a ride is denoted by `cost_duration_r2`.

<img src = "Exercise Images/Cost vs Duration.png" width = "600" style = "margin:auto"/>

The correlation between amount paid for a tide & the distance travelled is denoted by `cost_distance_r`. The amount of variability in the amount paid for a ride described by the distance travelled is denoted by `cost_distance_r2`.

<img src = "Exercise Images/Cost vs Distance.png" width = "600" style = "margin:auto"/>